In [2]:
from pycalphad import Model, Database
from pycalphad.core.autograd_utils import build_functions as bf_interpreted
from pycalphad.core.custom_ufuncify import ufuncify
from pycalphad.core.sympydiff_utils import build_functions as bf_compiled
import pycalphad.variables as v
from sympy.printing.ccode import CCodePrinter
import numpy as np

dbf = Database('2016-04-09-AlNi.tdb')
mod = Model(dbf, ['AL', 'NI', 'VA'], 'FCC_L12')

In [3]:
%lprun -f CCodePrinter._print_Piecewise obj, grad, hess = bf_compiled(mod.ast, tuple([v.T] + mod.site_fractions))
temps = np.linspace(300., 2000., 1000)

In [3]:
%time res1 = hess(300, 0.4, 0.6, 0.4, 0.6, 0.4, 0.6, 0.4, 0.6, 1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 220 µs


In [4]:
%time ag_obj, ag_grad, ag_hess = bf_interpreted(mod.ast, [v.T] + mod.site_fractions)

CPU times: user 303 ms, sys: 2 ms, total: 305 ms
Wall time: 305 ms


In [5]:
%time res2 = ag_hess(300, 0.4, 0.6, 0.4, 0.6, 0.4, 0.6, 0.4, 0.6, 1)

CPU times: user 554 ms, sys: 1 ms, total: 555 ms
Wall time: 554 ms


In [6]:
np.testing.assert_allclose(np.squeeze(res1), np.squeeze(res2))